In [ ]:
import matplotlib.pyplot as plt
import random 
import numpy as np

grid_length = 100 

def random_int():
    return  random.randint(-1, 1) #2*int(2*np.random.random())-1 
def random_terrain():
    # Parameters
    x_length = 100 
    y_length = 100 
    num_steps = 100000
    step_size = 0.1

    # Initialize the particle position
    x, y = 0, 0
    x_positions = [x]
    y_positions = [y]

    # Simulate Brownian motion
    for _ in range(num_steps):
        
        x_random_step = 2*int(2*np.random.random())-1 
        y_random_step = 2*int(2*np.random.random())-1  
        x += x_random_step 
        y += y_random_step 
        x_positions.append(x)
        y_positions.append(y)

    # Create a 2D histogram (heatmap)
    heatmap, xedges, yedges = np.histogram2d(x_positions, y_positions, bins=grid_length )
    
    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
    return heatmap 

# Plot the path and heatmap
plt.figure(figsize=(10, 8))

# Plot the heatmap
#plt.imshow(heatmap.T, extent=extent, origin='lower', cmap='hot', alpha=0.5)
#plt.colorbar(label='Density')

### Langevin Dynamics ### 
def grad(potential, x, dx ):
    x_length = len(potential)
    y_length = len(potential[0])
    dUdx = potential[(int(x[0])+1) % x_length][x[1] % y_length] - potential[x[0] % x_length][x[1] % y_length]
    dUdy = potential[x[0] % x_length ][(x[1]+1) % y_length] - potential[x[0] % x_length][x[1] % y_length]
    return np.array([dUdx, dUdy])

def langevin_path(terrain):
    # Parameters
    x_length = len(terrain)
    y_length = len(terrain[0])
    
    num_steps = 1000
    time_step = 1
    alpha = 1
    kappa = 0
    dx = 1 

    # Initialize the particle position
    x = np.array([int(grid_length/2),int(grid_length/2)])
    v = np.array([0.0,0.0]) 

    v_list = [v]  
    x_list = [x] 

    # Simulate Brownian motion
    for _ in range(num_steps):
        #x += (kappa * grad(terrain, x, dx) + alpha * np.array([random_int() for i in range(2)])) #* time_step 
        #v_temp = v * time_step 
        x += np.array([random_int() for i in range(2)]) 
        #x[0] += int(v_temp[0])
        #x[1] += int(v_temp[1])
        
        #print(x)
        x[0] = x[0] % x_length 
        x[1] = x[1] % y_length
        #print(x)
        x_temp = np.array([i for i in x])
        v_temp = np.array([i for i in v])
        v_list.append(v_temp)
        x_list.append(x_temp) 
        #print(x)
        #print(x_list)

    
    return x_list 

terrain = random_terrain()
terrain = 10 * terrain/ np.amax(terrain)
#positions = [[ [i,j] for j in range(50)] for i in range(50) ] 
#grad_field = [ grad(terrain, x, 1) for x in positions[0] ]

x_list = langevin_path(terrain)


plt.imshow(terrain.T, origin='lower', cmap='hot', alpha=0.5)
plt.colorbar(label='Density')

x_positions = [ _[0] for _ in x_list  ]
y_positions = [ _[1] for _ in x_list  ]
# Plot the path
plt.plot(x_positions, y_positions, 'bo', lw=1)
plt.plot(x_positions[-1], y_positions[-1], 'ro')  # Mark the final position


plt.title('Brownian Motion Path with Heatmap')
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import random 

In [ ]:
terrain = random_terrain()
print(len(terrain), len(terrain[0]))

In [ ]:
def pad_with(vector, pad_width, iaxis, kwargs):
    pad_value = kwargs.get('padder', 3)
    vector[:pad_width[0]] = pad_value
    vector[-pad_width[1]:] = pad_value
def padded_terrain(terrain):
    return np.pad(terrain, 1, pad_with)

a = np.arange(6)
a = a.reshape((2, 3))

padded_terrain(a)

In [ ]:
#working implementation 
step_size = 1
max_value = 1
grid_length = 20
def pad_with(vector, pad_width, iaxis, kwargs):
    pad_value = kwargs.get('padder', max_value )
    vector[:pad_width[0]] = pad_value
    vector[-pad_width[1]:] = pad_value
def padded_terrain(terrain):
    return np.pad(terrain, step_size, pad_with)

def random_terrain(grid_length):
    # Parameters
    x_length = 50
    y_length = 50
    num_steps = 100000
    step_size = 0.1

    # Initialize the particle position
    x, y = 0, 0
    x_positions = [x]
    y_positions = [y]

    # Simulate Brownian motion
    for _ in range(num_steps):
        
        x_random_step = 2*int(2*np.random.random())-1 
        y_random_step = 2*int(2*np.random.random())-1  
        x += x_random_step 
        y += y_random_step 
        x_positions.append(x)
        y_positions.append(y)

    # Create a 2D histogram (heatmap)
    heatmap, xedges, yedges = np.histogram2d(x_positions, y_positions, bins=grid_length )
    
    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
    heatmap = heatmap/np.amax(heatmap)
    return heatmap 

def mask(x,y):
    l= step_size
    weight = max_value * 1
    dx = weight if x[0]-l < y[0] < x[0]+l else 0 
    dy = weight if x[1]-l < y[1] < x[1]+l else 0 
    return dx*dy 
def weight_function(x):
    k = 1
    #ret = 0 if x >= max_value else np.exp(-k*x)
    return  np.exp(-k*x)

def update_terrain(terrain, x):
    terrain_new = [[0 for j in row] for row in terrain]
    for i,row in enumerate(terrain):
        for j,col in enumerate(row): 
            terrain_new[i][j] = min(max_value - 0 * 0.01 , terrain[i][j] + mask(x,[i,j] ))
            
    return np.array(terrain_new) 

def update_position(terrain, position):
    x_length = grid_length 
    y_length = grid_length  
    l = step_size 
    pad_terrain = padded_terrain(terrain)
    x_0 = position[0]
    x_1 = position[1]
    considered_tiles =[]
    for i in range(3):
        for j in range(3):
            if np.abs((i-1) + (j-1)) == 1: #i !=1 or j !=1:
                if 0 <= x_0 + (i-1)*l < grid_length and 0 <= x_1 + (j-1)*l < grid_length :
                    considered_tiles.append([x_0 + (i-1)*l, x_1 + (j-1)*l])
    ext_considered_tiles = []
    
    for i in [-2, -1, 0, 1, 2]:
        for j in [-2, -1, 0, 1, 2]:
            if np.abs(i) == 2 or np.abs(j) == 2:
                if 0 <= x_0 + i < grid_length and 0 <= x_1 + j < grid_length :
                    ext_considered_tiles.append([x_0 + i, x_1 + j ])
    #weights = [weight_function(pad_terrain[1+tile[0]][1+tile[1]]) for tile in considered_tiles ]
    weights = [ max_value - pad_terrain[1+tile[0]][1+tile[1]] for tile in considered_tiles ]
    ext_weights = [ max_value - terrain[tile[0]][tile[1]] for tile in ext_considered_tiles ]
    
    
    chosen_tile = random.choices(considered_tiles, weights, k = 1)[0] if sum(weights) > 0 else [0,0] #considered_tiles[0]
    #chosen_tile =  max(enumerate(considered_tiles), key= lambda x: pad_terrain[l+x[1][0]][l+x[1][1]])[1] 
    if sum(weights) == 0: 
        print("weights is zero")
        chosen_tile = random.choices(ext_considered_tiles, ext_weights, k=1)[0] if sum(ext_weights)>0 else ext_considered_tiles[0]
    #ext_chosen_tile = random.choices(ext_considered_tiles, ext_weights, k=1)[0] if sum(ext_weights) == 0 and chosen_tile == False else ext_considered_tiles[0]
    return chosen_tile 
   
def simulation(terrain_initial, position_initial, iterations):
    number_updates = iterations
    position_list = [position_initial]
    position = position_initial
    terrain = terrain_initial 
    for i in range(number_updates):
        position_old = position 
        
        position = update_position(terrain, position_old)
        
        terrain = update_terrain(terrain, position_old)
        position_list.append(position)
    
    return position_list, terrain 

def monkey_simulation(terrain_initial, position_initial):
    return 0 

terrain = np.round(2 * random_terrain(grid_length ))

optimal_iterations = int((grid_length/(2*step_size))**2)

simulation_result = simulation(terrain, [0,0], optimal_iterations) 

x_list = simulation_result[0]
final_terrain = simulation_result[1]


plt.imshow(final_terrain.T, origin='lower', cmap='hot', alpha=0.7)
plt.colorbar(label='Density')

x_positions = [ _[0] for _ in x_list  ]
y_positions = [ _[1] for _ in x_list  ]
# Plot the path
plt.plot(x_positions, y_positions, 'b-', lw=1)
plt.plot(x_positions[-1], y_positions[-1], 'ro')  # Mark the final position


plt.title('Brownian Motion Path with Heatmap')
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.grid(True)
plt.show()


In [ ]:
plt.imshow(terrain.T, origin='lower', cmap='hot', alpha=0.5)
plt.colorbar(label='Density')

plt.title('Brownian Motion Path with Heatmap')
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.grid(True)
plt.show()

In [ ]:
terrain_new = random_terrain(40)
terrain_new = 0 * terrain_new
plt.imshow(terrain_new.T, origin='lower', cmap='hot', alpha =1)

plt.show()

In [ ]:
#Mollified version 
step_size = 1
max_value = 1
grid_length = 25
def pad_with(vector, pad_width, iaxis, kwargs):
    pad_value = kwargs.get('padder', max_value )
    vector[:pad_width[0]] = pad_value
    vector[-pad_width[1]:] = pad_value
def padded_terrain(terrain):
    return np.pad(terrain, step_size, pad_with)

def random_terrain(grid_length):
    # Parameters
    x_length = 50
    y_length = 50
    num_steps = 100000
    step_size = 0.1

    # Initialize the particle position
    x, y = 0, 0
    x_positions = [x]
    y_positions = [y]

    # Simulate Brownian motion
    for _ in range(num_steps):
        
        x_random_step = 2*int(2*np.random.random())-1 
        y_random_step = 2*int(2*np.random.random())-1  
        x += x_random_step 
        y += y_random_step 
        x_positions.append(x)
        y_positions.append(y)

    # Create a 2D histogram (heatmap)
    heatmap, xedges, yedges = np.histogram2d(x_positions, y_positions, bins=grid_length )
    
    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
    heatmap = heatmap/np.amax(heatmap)
    return heatmap 

def mask(x,y):
    l= step_size
    weight = max_value * 0.6
    dx = weight if x[0]-l < y[0] < x[0]+l else 0 
    dy = weight if x[1]-l < y[1] < x[1]+l else 0 
    return dx*dy 
def distance(x,y):
    return np.abs(x[0]-y[0]) + np.abs(x[1]-y[1])
def weight_function(x):
    k = 1
    #ret = 0 if x >= max_value else np.exp(-k*x)
    return  np.exp(-k*x)

def update_terrain(terrain, x):
    terrain_new = [[0 for j in row] for row in terrain]
    for i,row in enumerate(terrain):
        for j,col in enumerate(row): 
            terrain_new[i][j] = min(max_value - 0 * 0.01 , terrain[i][j] + mask(x,[i,j] ))
            
    return np.array(terrain_new) 

def update_position(terrain, position, position_list):
    x_length = grid_length 
    y_length = grid_length  
    l = step_size 
    pad_terrain = padded_terrain(terrain)
    x_0 = position[0]
    x_1 = position[1]
    considered_tiles =[]
    for i in range(3):
        for j in range(3):
            if np.abs((i-1) + (j-1)) == 1: #i !=1 or j !=1:
                if 0 <= x_0 + (i-1)*l < grid_length and 0 <= x_1 + (j-1)*l < grid_length :
                    considered_tiles.append([x_0 + (i-1)*l, x_1 + (j-1)*l])
    ext_considered_tiles = []
    
    for i in [-2, -1, 0, 1, 2]:
        for j in [-2, -1, 0, 1, 2]:
            if np.abs(i) == 2 or np.abs(j) == 2:
                if 0 <= x_0 + i < grid_length and 0 <= x_1 + j < grid_length :
                    ext_considered_tiles.append([x_0 + i, x_1 + j ])
    #weights = [weight_function(pad_terrain[1+tile[0]][1+tile[1]]) for tile in considered_tiles ]
    weights = [ max_value - pad_terrain[1+tile[0]][1+tile[1]] for tile in considered_tiles ]
    ext_weights = [ max_value - terrain[tile[0]][tile[1]] for tile in ext_considered_tiles ]
    
    #random.shuffle(considered_tiles) 
    #random.shuffle(ext_considered_tiles)
    
    chosen_tile = random.choices(considered_tiles, weights, k = 1)[0] if sum(weights) > 0 else [0,0] #considered_tiles[0]
    chosen_tile =  max(enumerate(considered_tiles), key= lambda x: max_value - pad_terrain[l+x[1][0]][l+x[1][1]])[1] 
    if chosen_tile in position_list: 
        candidates = []
        for i in range(len(terrain)):
            for j in range(i):
                #print([i,j], position_list)
                if [i,j] not in position_list : 
                    candidates.append([[i,j], distance(position, [i,j])])
        
        chosen_tile = min(candidates, key = lambda x: x[1])[0] 
        print(candidates)
        print('exception', chosen_tile)
        #chosen_tile  =  max(enumerate(ext_considered_tiles), key= lambda x: max_value - pad_terrain[x[1][0]][x[1][1]])[1] 
        #chosen_tile = random.choices(ext_considered_tiles, ext_weights, k=1)[0] if sum(ext_weights)>0 else ext_considered_tiles[0]
    #ext_chosen_tile = random.choices(ext_considered_tiles, ext_weights, k=1)[0] if sum(ext_weights) == 0 and chosen_tile == False else ext_considered_tiles[0]
    return chosen_tile 
   
def simulation(terrain_initial, position_initial, iterations):
    number_updates = iterations
    position_list = [position_initial]
    position = position_initial
    terrain = terrain_initial 
    for i in range(number_updates):
        position_old = position 
        
        position = update_position(terrain, position_old, position_list)
        
        terrain = update_terrain(terrain, position_old)
        position_list.append(position)
    
    return position_list, terrain 

def monkey_simulation(terrain_initial, position_initial):
    return 0 

L = [random_terrain(grid_length) for i in range(1)]
summe = L[0]
for i in L[1:]:
    summe+= i 
terrain = (summe ) / (len(L))
#terrain = 0.5*(1-random_terrain(grid_length)) #np.round(3* (1-random_terrain(grid_length )))

optimal_iterations =   300 #int((grid_length/(2*step_size - 1))**2)

simulation_result = simulation(terrain, [0,0], optimal_iterations) 

x_list = simulation_result[0]
final_terrain = simulation_result[1]


plt.figure(figsize=(10, 10))

plt.imshow(final_terrain.T, origin='lower', cmap='bone', alpha=0.9)
plt.colorbar()

x_positions = [ _[0] for _ in x_list  ]
y_positions = [ _[1] for _ in x_list  ]
# Plot the path

plt.plot(x_positions, y_positions, 'b-', lw=1)
plt.plot(x_positions[-1], y_positions[-1], 'ro')  # Mark the final position


#plt.title('Brownian Motion Path with Heatmap')
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.grid(True)
plt.savefig('plot.png')
plt.show()




In [ ]:
plt.imshow(terrain.T, origin='lower', cmap='hot', alpha=0.5)
plt.colorbar(label='Density')

plt.title('Brownian Motion Path with Heatmap')
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.grid(True)
plt.show()

In [ ]:
def find_duplicates(list):
    A = []
    for item in list:
        if item in A:
            return item
        
    return 0 


In [ ]:
import numpy as np 
array = np.load('Kursk_4_200x200.npy', allow_pickle=True)
array  = np.sum(array, axis=2)

plt.imshow(array.T, origin='lower', cmap='hot', alpha=0.7)